In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os

# Configuration
MODEL_PATH = "attendance_model.h5"
LABELS_PATH = "labels.txt"
IMG_SIZE = (224, 224)

# Load model and labels
if not os.path.exists(MODEL_PATH) or not os.path.exists(LABELS_PATH):
    print("Error: Model or label file not found.")
    exit()

print("Loading model...")
model = tf.keras.models.load_model(MODEL_PATH)

with open(LABELS_PATH, 'r') as f:
    class_names = [line.strip() for line in f.readlines()]

print("System ready.")

# Initialize webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Mirror frame
    frame = cv2.flip(frame, 1)
    
    # Detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(60, 60))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        try:
            # Preprocess image
            face_roi = frame[y:y+h, x:x+w]
            rgb_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(rgb_face, IMG_SIZE)
            processed = preprocess_input(resized)
            
            # Predict
            input_tensor = np.expand_dims(processed, axis=0)
            predictions = model.predict(input_tensor, verbose=0)
            
            index = np.argmax(predictions)
            confidence = np.max(predictions)
            name = class_names[index]
            
            # Display results
            if confidence > 0.7:
                text = f"{name} ({int(confidence*100)}%)"
                color = (0, 255, 0)
            else:
                text = "Unknown"
                color = (0, 0, 255)
            
            cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
        except Exception as e:
            pass

    cv2.imshow("Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Loading model...


System ready.
